In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import dateutil.relativedelta
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import dask_xgboost as xgb
import dask.dataframe as dd
from sklearn import preprocessing, metrics
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
import gc
import time
import datetime
from  datetime import timedelta
from typing import Union
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from tqdm.notebook import tqdm_notebook as tqdm
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error
from typing import Union
import math
from tqdm.notebook import tqdm_notebook as tqdm
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

As we are dealing with large dataset, with the help of Kagglers we used the below function to reduce the memory usage

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: 
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

Loading data set

In [ ]:
data_dir = "../input/m5-forecasting-accuracy/"
calendar = pd.read_csv(data_dir+'calendar.csv')
calendar = reduce_mem_usage(calendar)
print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))

sell_prices = pd.read_csv(data_dir+'sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)
print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))

sales = pd.read_csv(data_dir+'sales_train_validation.csv')
print('Sales train validation has {} rows and {} columns'.format(sales.shape[0], sales.shape[1]))

In [ ]:
calendar.head()

In [ ]:
sell_prices.head()

In [ ]:
sales.head()

In [ ]:
date_column = [col for col in sales if col.startswith('d_')]
date_column[:5]

In [ ]:
#Calcuating all the sales for each row by (adding upto 1913 days)
sales['overall_sales'] = sales[date_column].sum(axis=1)
sales.groupby('state_id').agg({"overall_sales":"sum"}).reset_index()

In [ ]:
#Percentage sales state wise
sales_data_state = sales.groupby('state_id').agg({"overall_sales":"sum"})/sales.overall_sales.sum() * 100

The bar plot shows the overall sales percentage for all the 3 States. (CA - approx(44%), TX - approx(28%), WI - approx(29%))

In [ ]:
percentage_sales= sales_data_state.reset_index()
#plot the overall sales
percentage_sales.plot(x='state_id', y='overall_sales', kind='bar', color='darkred') 
plt.show()

The bar plot shows the overall sales percentage for all the stores

In [ ]:
# The percentage of sales for all the stores has been calculated 
store_data_sales=sales.groupby('store_id').agg({"overall_sales":"sum"})/sales.overall_sales.sum() * 100
#Plotting the sales for all the stores
store_data_sales = store_data_sales.reset_index()
store_data_sales.plot('store_id', 'overall_sales', kind='bar', color='darkred')

The bar plot shows the sales for each category

In [ ]:
#The percentage of sales based on the categories
category_sales_data = sales.groupby('cat_id').agg({"overall_sales":"sum"})/sales.overall_sales.sum() * 100
category_sales_data = category_sales_data.reset_index()
#Plotting the sales for each category
category_sales_data.plot('cat_id', 'overall_sales', kind='bar',color='darkred')

In [ ]:
start_date = datetime.datetime(2011,1,29)
dept_cat_level = sales.groupby(["state_id","cat_id"])[date_column].sum().reset_index().set_index(["state_id","cat_id"]).T
dept_cat_level['datum'] = [start_date + datetime.timedelta(days=x) for x in range(1913)]
dept_cat_level.set_index('datum', drop=True, inplace=True)
dept_cat_level.sort_index(inplace=True)
dept_cat_level.columns = [f'{i}_{j}' if j != '' else f'{i}' for i,j in dept_cat_level.columns]
dept_cat_monthly_level = dept_cat_level.groupby(pd.Grouper(freq='1M')).sum()
dept_cat_monthly_level.head()

To analyze the trends (yearly) based on Categories for all the three states

The sales for all the three states hikes at the start, but CA keeps incresing in comparitice to other states. Texas had a hike at the start but remained almost static in the following years (from 2013).

**Category - Food**

In [ ]:
plt.plot( dept_cat_monthly_level.index.values, 'CA_FOODS', data=dept_cat_monthly_level, color='blue', linewidth=2)
plt.plot( dept_cat_monthly_level.index.values, 'TX_FOODS', data=dept_cat_monthly_level, color='green', linewidth=2)
plt.plot( dept_cat_monthly_level.index.values, 'WI_FOODS', data=dept_cat_monthly_level, color='red', linewidth=2)
plt.legend()

**Category - Hobbies**

In [ ]:
plt.plot( dept_cat_monthly_level.index.values, 'CA_HOBBIES', data=dept_cat_monthly_level, color='blue', linewidth=2)
plt.plot( dept_cat_monthly_level.index.values, 'TX_HOBBIES', data=dept_cat_monthly_level, color='green', linewidth=2)
plt.plot( dept_cat_monthly_level.index.values, 'WI_HOBBIES', data=dept_cat_monthly_level, color='red', linewidth=2)
plt.legend()

Category - Household

In [ ]:
plt.plot( dept_cat_monthly_level.index.values, 'CA_HOUSEHOLD', data=dept_cat_monthly_level, color='blue', linewidth=2)
plt.plot( dept_cat_monthly_level.index.values, 'TX_HOUSEHOLD', data=dept_cat_monthly_level, color='green', linewidth=2)
plt.plot( dept_cat_monthly_level.index.values, 'WI_HOUSEHOLD', data=dept_cat_monthly_level, color='red', linewidth=2)
plt.legend()

Model ready data

In [ ]:
idCols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
product = sales[idCols].drop_duplicates()

In [ ]:
submission = pd.read_csv(data_dir+'sample_submission.csv')
validate_submission = submission[submission.id.str.endswith('validation')]
eval_submission = submission[submission.id.str.endswith('evaluation')]
newcolumns = ["id"] + ["d_{}".format(i) for i in range(1914, 1914+28)]
validate_submission.columns = newcolumns
validate_submission = validate_submission.merge(product, how = 'left', on = 'id')


In [ ]:
idCols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# Use only the 2 three years data
DAYS = 365*2; end=1913
dayCols = ["d_{}".format(i) for i in range(end-DAYS+1, end+1)]
print(len(dayCols), dayCols[0])
sales = sales[idCols+dayCols]
print(sales.shape)
sales.head()

We use melt function for calculating demand

In [ ]:
def melted(df, name=""):
    df = pd.melt(df, id_vars = idCols, var_name = 'day', value_name = 'demand')
    print('{}: {} rows and {} columns'.format(name, df.shape[0], df.shape[1]))
    df = reduce_mem_usage(df)
    return df
mel_sale = melted(sales)
mel_sale["part"] = "train"
mel_val = melted(validate_submission)
mel_val["part"] = "validate"


In [ ]:
data = pd.concat([mel_sale, mel_val], axis = 0)
data.head()

We used garbage collection to delete thew unused variables, it helps to reduce the use of memory

In [ ]:
del mel_sale, mel_val
del submission, validate_submission, eval_submission, product
del sales,
gc.collect()

In [ ]:
calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)
print('Our dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

# get the sell price data
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'])
print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

In [ ]:
del calendar, sell_prices
gc.collect()

In [ ]:
def encode_categorical(dt, cols):
    for col in cols:
        dt[col] = dt[col].astype("category").cat.codes.astype("int16")
        dt[col] -= dt[col].min()
    return dt


data = encode_categorical(data, ["cat_id", "dept_id", "item_id", "state_id", "store_id"]).pipe(reduce_mem_usage)
values = {'event_name_1': "normal", 'event_type_1': "normal", "event_name_2": "normal", 'event_type_2': "normal"}
data.fillna(value=values, inplace = True);
data = encode_categorical(data, ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]).pipe(reduce_mem_usage)

In [ ]:
def datetime_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    attrs = ["year","quarter","month","week","day","dayofweek","weekday","weekofyear",]
    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df['date'].dt, attr).astype(dtype)
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    return df

data = datetime_features(data)
data.head()

In [ ]:
data.sort_values(by=['id', "date"], inplace=True)

X_train = data[data["part"]=="train"]
X_val = data[data["part"]=="validate"]
X_eval = data[data["part"]=="evaluate"]
del data; gc.collect()

In [ ]:
X_train_store = X_train
X_val_store = X_val

In [ ]:
X_train['lag_t28'] = X_train.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
X_train['rolling_mean_t7'] = X_train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
X_train['rolling_std_t7'] = X_train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
X_train['rolling_mean_t30'] = X_train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
X_train['rolling_std_t30'] = X_train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
X_train['rolling_mean_t90'] = X_train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
X_train['rolling_std_t90'] = X_train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())

In [ ]:
X_train.dropna(inplace = True)
gc.collect()

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "part", "date", "demand","d", "wm_yr_wk", "weekday"]
train_cols = X_train.columns[~X_train.columns.isin(useless_cols)]
train_cols 

y_train = X_train["demand"]
X_train = X_train[train_cols]

In [ ]:
np.random.seed(777)
X, x_test, Y, y_test = train_test_split(X_train, y_train, test_size=0.2)

Best values have been chosen as parameters (3 other trials were performed before choosing this value)

In [ ]:
params = {'num_leaves': 475,
          'min_child_weight': 0.4,
          'feature_fraction': 0.6,
          'bagging_fraction': 0.5,
          'min_data_in_leaf': 100,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.075,
          "boosting_type": "gbdt",
          "bagging_seed": 5,
          "metric": 'rmse',
          "verbosity": -1,
          'reg_alpha': 0.45,
          'reg_lambda': 0.6,   
          'random_state': 200,
         }

In [ ]:
train_set = lgb.Dataset(X, Y)
test_set = lgb.Dataset(x_test, y_test)
del X, Y, x_test, y_test; gc.collect()

In [ ]:
model = lgb.train(params, train_set, valid_sets = [test_set], verbose_eval = 100)

In [ ]:
import datetime
import time
X_train = X_train_store
X_val = X_val_store
lastdate = X_train["date"].max() - pd.DateOffset(days=90)
X_train = X_train[X_train['date'] >= lastdate]

In [ ]:
import datetime
import time
def predict(model, X_train, X_test, factor=1):
    DATES = X_test["date"].unique()
    no_date = len(DATES)
    col = ["id"] + ["F{}".format(i) for i in range(1, no_date+1)]
    itemId = X_train["dept_id"].unique()
    items = []
    itemId = sorted(itemId)
    for item_id in itemId:
        test = X_test[X_test["dept_id"]==item_id]
        ids = test["id"].unique()
        id_na = np.zeros((len(ids), no_date+1))
        data_col = pd.DataFrame(id_na, columns=col)
        data_col["id"] = test[test["date"]==DATES[0]]["id"].values
        train = X_train[X_train["dept_id"]==item_id]
        lastmonth = pd.to_datetime(train.head(1)["date"])
        for idx, date in enumerate(DATES):
            newrow = test[test["date"]==date]
            train = train.append(newrow)
            train.sort_values(by=['id', "date"], inplace=True)
            train['lag_t28'] = train.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
            train['rolling_mean_t7'] = train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
            train['rolling_std_t7'] = train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
            train['rolling_mean_t30'] = train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
            train['rolling_std_t30'] = train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
            train['rolling_mean_t90'] = train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
            train['rolling_std_t90'] = train.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
            feat=train
            x = feat.loc[feat["date"] == date , train_cols]
            val_pred = model.predict(x, num_iteration=model.best_iteration)
            data_col[f"F{idx+1}"] = val_pred*factor
            train.loc[train["date"]==date, "demand"] = val_pred*factor
            lastmonth = lastmonth + pd.DateOffset(days=1)
            train = train[train['date'] >= str(lastmonth.values[0])]
        items.append(data_col)
        items = [pd.concat(items)]
        #print(item_id)
    items = pd.concat(items)
    return items

In [ ]:
import datetime
import time
predict_data = []
weights = [1, 1.02, 1.023, 1.015]
for w in weights:
    pred = predict(model, X_train, X_val, factor=w)
    predict_data.append(pred)

In [ ]:
import datetime
import time
avg_predict = pd.DataFrame([])
avg_predict["id"] = predict_data[0]["id"]
for i in range(1, 29):
    avg_predict[f"F{i}"] = (predict_data[1][f"F{i}"]+predict_data[2][f"F{i}"]+predict_data[3][f"F{i}"])/3

In [ ]:
submission = pd.read_csv(data_dir+'sample_submission.csv')
data_evaluation = submission[submission.id.str.endswith('evaluation')]
assert len(data_evaluation)==len(avg_predict)

In [ ]:
def save_csv(i, d):
    df = pd.concat([d, data_evaluation]) 

    df.sort_values("id", inplace = True)
    #df.reset_index(drop=True, inplace = True, index=False)
    
    if i=="":
        df.to_csv(f"submission.csv")
    else:
        df.to_csv(f"submission.v1.{i}.csv")
    
for i in range(len(predict_data)):
    save_csv(predict_data[i])

save_csv("", avg_predict)

Feature Importance

In [ ]:
feature_imp = pd.DataFrame(zip(model.feature_importance(importance_type='gain', iteration=-1), model.feature_name()),  columns=['Value_Gain','Feature'])
feature_imp['Feature_importance'] = model.feature_importance(importance_type='split', iteration=-1)

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x="Feature_importance", y="Feature", color='darkred',data=feature_imp.sort_values(by="Feature_importance", ascending=False))
plt.title('LightGBM Features (Split)')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x="Feature_importance", y="Feature", color='darkred',data=feature_imp.sort_values(by="Feature_importance", ascending=False))
plt.title('LightGBM Features (Split)')
plt.tight_layout()
plt.show()